In [26]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, AveragePooling2D ,MaxPool2D, Dense, Dropout, Flatten, BatchNormalization
from tensorflow.keras.optimizers import SGD,Adam
from tensorflow.keras import regularizers
import cv2
import os
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.preprocessing import LabelEncoder
import matplotlib.pyplot as plt
from sklearn.ensemble import BaggingClassifier
# from keras.wrappers.scikit_learn import KerasClassifier

In [27]:
label = {
    0: 'apple',
    1: 'banana',
    2: 'grape',
    3: 'mango',
    4: 'strawberry',
}

train_dir = '/kaggle/input/dataset11/dataset/dataset/train'
test_dir = '/kaggle/input/dataset11/dataset/dataset/test'


training_data = []

In [28]:


# # Preprocessing step

#data augmentation 
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
) # pass your custom function here

test_datagen = ImageDataGenerator(rescale=1./255) # pass your custom function here

In [29]:

def is_image_file(file):
    return file.lower().endswith(('.png', '.jpg', '.jpeg'))

In [30]:
def create():
    total = 9900
    training_data = []

    for i in range(1, 6):
        pa = os.path.join(train_dir, str(i))
        files = [file for file in os.listdir(pa) if is_image_file(file)]

        for file in files:
            total -= 1
            path = os.path.join(pa, file)
            img_data = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
            img_data = cv2.resize(img_data, (224, 224))
            img_data = img_data.reshape(-1, 224, 224, 1)
            img_data = img_data / 255.0

            if total % 2 == 0:
                img_ar = next(train_datagen.flow(img_data, batch_size=1))
                training_data.append([np.array(img_ar), label[i - 1]])

            training_data.append([np.array(img_data), label[i - 1]])

    shuffle(training_data)
    return training_data

In [31]:
def create_test_predict():
    total = 100
    test_data = []

    files = [file for file in os.listdir(test_dir) if is_image_file(file)]

    for file in files:
        total -= 1
        path = os.path.join(test_dir, file)
        img_data = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
        img_data = cv2.resize(img_data, (224, 224))
        img_data = img_data.reshape(-1, 224, 224, 1)

        if total % 2 == 0:
            img_ar = next(test_datagen.flow(img_data, batch_size=1))
            test_data.append([np.array(img_ar)])

        test_data.append([np.array(img_data)])

    shuffle(test_data)
    return test_data

In [32]:
def create_test_predict(test_dir):
    test_data = []

    files = [file for file in os.listdir(test_dir) if is_image_file(file)]

    for file in files:
        path = os.path.join(test_dir, file)
        img_data = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
        img_data = cv2.resize(img_data, (224, 224))
        img_data = img_data.reshape(-1, 224, 224, 1)

        test_data.append([np.array(img_data)])

    shuffle(test_data)
    return test_data




In [33]:
# Assuming 'test_directory' is the path to your test image directory or test file
test_directory = '/kaggle/input/dataset11/dataset/dataset/test'  # Replace with the path to your actual test directory or file

prediction_test = create_test_predict(test_directory)

X_test = np.array([i[0] for i in prediction_test]).reshape(-1, 224, 224, 1)
print(len(X_test))

FileNotFoundError: [Errno 2] No such file or directory: '/kaggle/input/dataset11/dataset/dataset/test'

In [ ]:
# ===========================================================================================================
train_data= create()

X_train = np.array([i[0] for i in train_data]).reshape(-1, 224, 224,1)
y_train = np.array([i[1] for i in train_data])


print(y_train.shape)
x_train, x_test, y_train, y_test = train_test_split(X_train, y_train, random_state=10, test_size=0.25,shuffle=True)
#================================================================================================================================================

In [ ]:
label_encoder = LabelEncoder()

integer_encoded = label_encoder.fit_transform(y_train)
integer_encoded_test=label_encoder.fit_transform(y_test)

In [ ]:
def alex_net():
    model = Sequential()

    model.add(Conv2D(filters=96, kernel_size=(11, 11), strides=(4, 4), padding='valid', input_shape=(224, 224, 1), activation='relu'))
    model.add(BatchNormalization())
    model.add(MaxPool2D(pool_size=(3, 3), strides=2))

    model.add(Conv2D(filters=256, kernel_size=(5, 5), strides=(1, 1), padding='valid', activation='relu'))
    model.add(BatchNormalization())
    model.add(MaxPool2D(pool_size=(3, 3), strides=2))

    model.add(Conv2D(filters=384, kernel_size=(3, 3), strides=(1, 1), padding='valid', activation='relu'))
    model.add(BatchNormalization())
    model.add(Conv2D(filters=384, kernel_size=(3, 3), strides=(1, 1), padding='valid', activation='relu'))
    model.add(BatchNormalization())
    model.add(Conv2D(filters=256, kernel_size=(3, 3), strides=(1, 1), padding='valid', activation='relu'))
    model.add(BatchNormalization())
    model.add(MaxPool2D(pool_size=(3, 3), strides=2))

    model.add(Flatten())
    model.add(Dense(units=4096, activation='relu',kernel_regularizer=regularizers.l2(0.01), bias_regularizer=regularizers.l2(0.010)))
    model.add(Dropout(rate=0.5))

    model.add(Dense(units=4096, activation='relu',kernel_regularizer=regularizers.l2(0.010), bias_regularizer=regularizers.l2(0.010)))
    model.add(Dropout(rate=0.5))

    model.add(Dense(units=5, activation='softmax'))
    zf=zfnet()
    model.add(zf)

    return model

In [ ]:

def zfnet():
    model = Sequential()

    # Convolutional layers
    model.add(Conv2D(96, (7, 7), strides=(2, 2), activation='relu', input_shape=(224,224,1)))
    model.add(MaxPool2D((3, 3), strides=(2, 2)))

    model.add(Conv2D(256, (5, 5), activation='relu'))
    model.add(MaxPool2D((3, 3), strides=(2, 2)))

    model.add(Conv2D(384, (3, 3), activation='relu'))
    model.add(Conv2D(384, (3, 3), activation='relu'))
    model.add(Conv2D(256, (3, 3), activation='relu'))
    model.add(MaxPool2D((3, 3), strides=(2, 2)))

    # Flatten layer
    model.add(Flatten())

    # Fully connected layers
    model.add(Dense(4096, activation='relu'))
    model.add(Dropout(0.3))

    model.add(Dense(4096, activation='relu',kernel_regularizer=regularizers.l2(0.0001), bias_regularizer=regularizers.l2(0.0001)))
    model.add(Dropout(0.3))

    model.add(Dense(units=5, activation='softmax'))

    return model

In [ ]:
def LeNet5():
    
    model = Sequential()
    
    model.add(Conv2D(filters=6 , kernel_size=(5,5) , strides=(1,1) , activation="tanh" , input_shape=(32 , 32,1 )))
              
    model.add(MaxPool2D((2,2)))
    
    model.add(Conv2D(filters=16 , kernel_size=(5,5) , strides=(1,1) , activation="tanh" ))   
                  
    model.add(MaxPool2D((2,2)))
              
    model.add(Flatten())
    
    model.add(Dense(units=120 , activation="tanh"))
    
    model.add(Dropout(0.2))
    
    model.add(Dense(units=84 , activation="tanh"))
    
    model.add(Dropout(0.2))
    
    model.add(Dense(units=10 , activation="softmax"))
    
    adam = Adam(learning_rate=0.01)
    
    model.compile(optimizer=adam , loss='sparse_categorical_crossentropy'  , metrics=['accuracy'])
    
    return model

In [ ]:
# vgg16
def vgg_net():
    model = Sequential()
    model.add(Conv2D(filters=32, kernel_size=(7, 7), strides=1, padding='valid', input_shape=(224, 224,1),
                     activation='relu'))
    model.add(Conv2D(filters=32, kernel_size=(5, 5), strides=1, activation='relu'))
    model.add(MaxPool2D(pool_size=(2, 2), strides=2))
   
    
    model.add(Conv2D(filters=64, kernel_size=(3, 3), strides=1, activation='relu'))
    model.add(Conv2D(filters=64, kernel_size=(3, 3), strides=1, activation='relu'))
    model.add(MaxPool2D(pool_size=(2, 2), strides=2))
   

    model.add(Conv2D(filters=96, kernel_size=(3, 3), strides=1, activation='relu'))
    model.add(Conv2D(filters=96, kernel_size=(3, 3), strides=1, activation='relu'))
    model.add(Conv2D(filters=96, kernel_size=(3, 3), strides=1, activation='relu'))
    model.add(MaxPool2D(pool_size=(2, 2), strides=2))
#     model.add(Dropout(0.5))
    
    model.add(Conv2D(filters=128, kernel_size=(3, 3), strides=1, activation='relu'))
    model.add(Conv2D(filters=128, kernel_size=(3, 3), strides=1, activation='relu'))
    model.add(Conv2D(filters=128, kernel_size=(3, 3), strides=1, activation='relu'))
    model.add(MaxPool2D(pool_size=(2, 2), strides=2))
    model.add(Dropout(0.2))
    
    model.add(Conv2D(filters=128, kernel_size=(3, 3), strides=1, activation='relu'))
    model.add(Conv2D(filters=128, kernel_size=(3, 3), strides=1, activation='relu'))
    model.add(Conv2D(filters=128, kernel_size=(3, 3), strides=1, activation='relu'))
    model.add(MaxPool2D(pool_size=(2, 2), strides=2))
#     model.add(Dropout(0.5))
    
    model.add(Dense(units=4096, activation='relu',kernel_regularizer=regularizers.l2(0.01), bias_regularizer=regularizers.l2(0.0001)))
    model.add(Dropout(0.3))
    model.add(Dense(units=4096, activation='relu',kernel_regularizer=regularizers.l2(0.0001), bias_regularizer=regularizers.l2(0.0001)))
    model.add(Dropout(0.3))
    model.add(Dense(units=1000, activation='softmax'))

    return model

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

class MultiHeadAttention(layers.Layer):
    def __init__(self, d_model, n_heads):
        super(MultiHeadAttention, self).__init__()
        self.n_heads = n_heads
        self.d_model = d_model

        self.query_dense = layers.Dense(d_model)
        self.key_dense = layers.Dense(d_model)
        self.value_dense = layers.Dense(d_model)
        self.final_dense = layers.Dense(d_model)

    def split_heads(self, x, batch_size):
        x = tf.reshape(x, (batch_size, -1, self.n_heads, self.d_model // self.n_heads))
        return tf.transpose(x, perm=[0, 2, 1, 3])

    def call(self, inputs):
        query, key, value, mask = inputs['query'], inputs['key'], inputs['value'], inputs['mask']
        batch_size = tf.shape(query)[0]

        # Linear layers
        query = self.query_dense(query)
        key = self.key_dense(key)
        value = self.value_dense(value)

        # Split heads
        query = self.split_heads(query, batch_size)
        key = self.split_heads(key, batch_size)
        value = self.split_heads(value, batch_size)

        # Scaled dot-product attention
        scaled_attention, attention_weights = self.scaled_dot_product_attention(query, key, value, mask)

        # Concatenate heads and apply final linear layer
        scaled_attention = tf.transpose(scaled_attention, perm=[0, 2, 1, 3])
        concat_attention = tf.reshape(scaled_attention, (batch_size, -1, self.d_model))
        output = self.final_dense(concat_attention)

        return output, attention_weights

    def scaled_dot_product_attention(self, query, key, value, mask):
        matmul_qk = tf.matmul(query, key, transpose_b=True)

        # Scale matmul_qk
        dk = tf.cast(tf.shape(key)[-1], tf.float32)
        scaled_attention_logits = matmul_qk / tf.math.sqrt(dk)

        # Add mask to the scaled tensor
        if mask is not None:
            scaled_attention_logits += (mask * -1e9)

        # Softmax is normalized on the last axis (seq_len_k) so that the scores
        # add up to 1.
        attention_weights = tf.nn.softmax(scaled_attention_logits, axis=-1)

        output = tf.matmul(attention_weights, value)

        return output, attention_weights

class PositionalEncoding(layers.Layer):
    def __init__(self, max_len, d_model):
        super(PositionalEncoding, self).__init__()
        self.encoding = self.positional_encoding(max_len, d_model)

    def get_angles(self, pos, i, d_model):
        angle_rates = 1 / tf.pow(10000, (2 * (i // 2)) / tf.cast(d_model, tf.float32))
        return pos * angle_rates

    def positional_encoding(self, position, d_model):
        angle_rads = self.get_angles(
            pos=tf.range(position, dtype=tf.float32)[:, tf.newaxis],
            i=tf.range(d_model, dtype=tf.float32)[tf.newaxis, :],
            d_model=d_model)
        # apply sin to even indices in the array; 2i
        sines = tf.math.sin(angle_rads[:, 0::2])
        # apply cos to odd indices in the array; 2i+1
        cosines = tf.math.cos(angle_rads[:, 1::2])

        pos_encoding = tf.concat([sines, cosines], axis=-1)
        pos_encoding = pos_encoding[tf.newaxis, ...]
        return tf.cast(pos_encoding, tf.float32)

    def call(self, inputs):
        return inputs + self.encoding[:, :tf.shape(inputs)[1], :]

class VisionTransformerBlock(layers.Layer):
    def __init__(self, d_model, n_heads, dff, dropout_rate=0.1):
        super(VisionTransformerBlock, self).__init__()
        self.attention = MultiHeadAttention(d_model, n_heads)
        self.ffn = keras.Sequential([
            layers.Dense(dff, activation='relu'),
            layers.Dense(d_model)
        ])
        self.layernorm1 = layers.LayerNormalization(epsilon=1e-6)
        self.layernorm2 = layers.LayerNormalization(epsilon=1e-6)
        self.dropout1 = layers.Dropout(dropout_rate)
        self.dropout2 = layers.Dropout(dropout_rate)

    def call(self, inputs, training, mask):
        attention_output, _ = self.attention({
            'query': inputs,
            'key': inputs,
            'value': inputs,
            'mask': mask
        })
        attention_output = self.dropout1(attention_output, training=training)
        out1 = self.layernorm1(inputs + attention_output)

        ffn_output = self.ffn(out1)
        ffn_output = self.dropout2(ffn_output, training=training)
        return self.layernorm2(out1 + ffn_output)

class VisionTransformer(keras.Model):
    def __init__(self, image_size, patch_size, num_layers, d_model, num_heads, dff, num_classes, rate=0.1):
        super(VisionTransformer, self).__init__()

        self.patch_size = patch_size
        self.num_patches = (image_size // patch_size) ** 2
        self.embedding_dim = d_model

        self.flatten = layers.Flatten()
        self.patch_embedding = layers.Conv2D(d_model, (patch_size, patch_size), strides=(patch_size, patch_size), activation='relu')
        self.positional_encoding = PositionalEncoding(self.num_patches, d_model)

        self.transformer_blocks = [VisionTransformerBlock(d_model, num_heads, dff) for _ in range(num_layers)]
        self.dropout = layers.Dropout(rate)

        self.classifier = layers.Dense(num_classes)

    def call(self, inputs, training):
        x = self.patch_embedding(inputs)
        x = self.flatten(x)
        x += self.positional_encoding(x)

        mask = tf.linalg.band_part(tf.ones((self.num_patches, self.num_patches)), -1, 0)
        mask = 1 - mask  # Lower triangular part of the matrix

        for transformer_block in self.transformer_blocks:
            x = transformer_block(x, training, mask)

        x = self.dropout(x, training=training)
        output = self.classifier(x)

        return output

# Example usage:
image_size = 224
patch_size = 16
num_layers = 6
d_model = 256
num_heads = 8
dff = 512
num_classes = 5
dropout_rate = 0.3

vit_model = VisionTransformer(image_size, patch_size, num_layers, d_model, num_heads, dff, num_classes, dropout_rate)

# You can compile and train the model using standard Keras methods.
# For example, using categorical crossentropy loss and the Adam optimizer:
vit_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
history=vit_model.fit(x_train, integer_encoded, epochs=50, batch_size=200, validation_data=(x_test, integer_encoded_test), callbacks=[early_stopping])

# Replace `your_image_data` and `your_labels` with your actual training data


In [ ]:

callbacks=[]

adam=Adam(learning_rate=0.00001)
sgd = SGD(learning_rate=0.001, momentum=0.9)

In [ ]:
from sklearn.ensemble import BaggingClassifier
from sklearn.base import BaseEstimator, ClassifierMixin
from keras.models import Model

# Your existing code for AlexNet
alex = alex_net()
alex.layers[0].trainable = False
early_stopping = tf.keras.callbacks.EarlyStopping(patience=8)
alex.compile(loss='sparse_categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])
history = alex.fit(x_train, integer_encoded, epochs=50, batch_size=200, validation_data=(x_test, integer_encoded_test), callbacks=[early_stopping])


In [ ]:
# Save the trained model
model_save_path = '/kaggle/working/alexnet_model.h5'
alex.save(model_save_path)

# Print a message indicating the save path
print(f"Model saved at: {model_save_path}")

In [ ]:
loaded_model = tf.keras.models.load_model('/kaggle/working/alexnet_model.h5')


In [ ]:
# Wrap the Keras model
plt.figure(figsize=(10, 5))

# Plot training & validation accuracy values
plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Model accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend(['Train', 'Validation'], loc='upper left')

# Plot training & validation loss values
plt.subplot(1, 2, 2)
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend(['Train', 'Validation'], loc='upper left')

plt.show()

In [ ]:
# Assuming 'prediction_test' is your test data
predictions = alex.predict(x_test)
predicted_labels = np.argmax(predictions, axis=1)

In [34]:
import os
import pandas as pd

data = {'image_id': [], 'label': []}

# Assuming 'x_test' is your test data
predictions = alex.predict(X_test)
predicted_labels = np.argmax(predictions, axis=1)

# Get the list of file names (without extensions) from the "test" directory
test_directory = '/kaggle/input/dataset11/dataset/dataset/test'
test_file_names = [os.path.splitext(file)[0] for file in os.listdir(test_directory) if is_image_file(file)]

# Update the 'image_id' column with the extracted file names
data['image_id'] = test_file_names
print(len(test_file_names))
print(len(predicted_labels))
# Update the 'label' column with the predicted labels
data['label'] = predicted_labels + 1  # Adding 1 to convert labels back to 1-indexing

df2 = pd.DataFrame(data)

csv_file_path = '/kaggle/working/alex.csv'
df2.to_csv(csv_file_path, index=False)

print(f"CSV file saved at: {csv_file_path}")

print(df2.head())




NameError: name 'alex' is not defined